# Environment

In [1]:
# LeafSim
from leafsim import LeafSim, SUPPORTED_MODELS

# Helper function for the toy example
from utils import apply_formatting

# Models currently supported by LeafSim
# from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

# Dataset used in the example
from sklearn.datasets import load_iris

# To perform train - test splits
from sklearn.model_selection import train_test_split

# Useful libraries
import numpy as np
import pandas as pd

# Useful function
from IPython.display import display

In [2]:
# Make the example reproducible
np.random.seed(42)

# Data

In [3]:
data = load_iris(as_frame=True)

In [4]:
# Features as pandas dataframe
X = data["data"]
# Target as pandas series
y = data["target"]

# Names of feature columns
feature_cols = data["feature_names"]
# Name of target column
target_col = y.name
# Name of prediction column
predicted_col = "Predicted" + target_col

In [5]:
# Split into train and test data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=46
)

In [6]:
X_train.shape[0], X_test.shape[0]

(120, 30)

# Model

In [7]:
# Currently supported models
# For your own custom model
# see end of notebook for an example
SUPPORTED_MODELS

['CatBoostClassifier',
 'CatBoostRegressor',
 'RandomForestClassifier',
 'RandomForestRegressor']

In [8]:
# Train model
model = RandomForestClassifier(
    random_state=46, n_jobs=-1, n_estimators=50, min_samples_leaf=4
)
model.fit(
    X_train,
    y_train,
)

RandomForestClassifier(min_samples_leaf=4, n_estimators=50, n_jobs=-1,
                       random_state=46)

In [9]:
# Store as series (just like y_train and y_test)
y_hat_train = pd.Series(model.predict(X_train))
y_hat_test = pd.Series(model.predict(X_test))

# Apply LeafSim

In [10]:
leafsim_instance = LeafSim(model)

In [11]:
top_n_ids, top_n_similarities = leafsim_instance.generate_explanations(
    X_train[feature_cols], X_test[feature_cols]
)

In [12]:
# One row for each observation in the test set
# By default, finding top 10 most similar observation
# in the training set
top_n_ids.shape

(30, 10)

In [13]:
# One row for each observation in the test set
# By default, the similarity scores of the top 10
# most similar observations
top_n_similarities.shape

(30, 10)

## Example where model performs poorly

In [14]:
def showcase(idx_to_explain: int):
    """
    :param idx_to_explain:
        Integer location of row in test feature dataframe and target series
    """
    # Let's explain the prediction of this test observation
    test_example = X_test.iloc[idx_to_explain].to_frame().T
    # Adding the target information
    test_example[predicted_col] = y_hat_test.iloc[idx_to_explain]
    test_example[target_col] = y_test.iloc[idx_to_explain]
    # Showcasing the observation
    display(test_example)

    # Show the main observations the model relies on
    # when prediction our examples
    top_n_explanations = top_n_ids[idx_to_explain]

    similar_obs_example = X_train.iloc[top_n_explanations].copy()
    similar_obs_example[f"{predicted_col}"] = y_hat_train.iloc[
        top_n_explanations
    ].values
    similar_obs_example[f"{target_col}"] = y_train.iloc[top_n_explanations]
    similar_obs_example["similarity"] = top_n_similarities[idx_to_explain]

    formatting = {
        c: lambda x: f"{x:,.1f}"
        for c in similar_obs_example.columns
    }
    display(
        apply_formatting(
            test_example,
            similar_obs_example.drop(columns=[predicted_col]),
            formatting
        )
    )

In [15]:
# Class for which the model performs poorly
idx_to_explain = np.argmax(y_test != y_hat_test.values)
showcase(idx_to_explain)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Predictedtarget,target
106,4.9,2.5,4.5,1.7,1,2


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,similarity
84,5.4,3.0,4.5,1.5,1.0,0.5
57,4.9,2.4,3.3,1.0,1.0,0.5
59,5.2,2.7,3.9,1.4,1.0,0.5
93,5.0,2.3,3.3,1.0,1.0,0.5
60,5.0,2.0,3.5,1.0,1.0,0.5
98,5.1,2.5,3.0,1.1,1.0,0.5
121,5.6,2.8,4.9,2.0,2.0,0.5
113,5.7,2.5,5.0,2.0,2.0,0.5
90,5.5,2.6,4.4,1.2,1.0,0.5
89,5.5,2.5,4.0,1.3,1.0,0.4


## Example where model performs well

In [16]:
# Class for which the model performs well
idx_to_explain = np.argmax(y_test == y_hat_test.values)
showcase(idx_to_explain)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Predictedtarget,target
4,5.0,3.6,1.4,0.2,0,0


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,similarity
3,4.6,3.1,1.5,0.2,0.0,1.0
30,4.8,3.1,1.6,0.2,0.0,1.0
17,5.1,3.5,1.4,0.3,0.0,1.0
29,4.7,3.2,1.6,0.2,0.0,1.0
22,4.6,3.6,1.0,0.2,0.0,1.0
40,5.0,3.5,1.3,0.3,0.0,1.0
46,5.1,3.8,1.6,0.2,0.0,1.0
2,4.7,3.2,1.3,0.2,0.0,1.0
7,5.0,3.4,1.5,0.2,0.0,1.0
0,5.1,3.5,1.4,0.2,0.0,1.0


## Custom model

Currently, LeafSim is not by default supporting XGBoost. To use it and any other ensemble model, one can modify the initialising of the LeafSim instance:

In [17]:
from xgboost import XGBRegressor

In [18]:
# Create a model wrapper instance
# The custom model really can be any type of instance
# that makes sense for LeafSim
# All that is requires is for it to have the attribute:
#   - get_leaf_indices (see next cell)
CustomModel = XGBRegressor

In [19]:
# Set the attribute "get_leaf_indices" that every custom model
# to be passed to LeafSim needs to have.
# This function takes in X (n_data, n_features)
# and provides a matrix of (n_data, n_predictions)
# where predictors are the individual predictors in the ensemble (trees)
# and the values are the leaf indices.
setattr(CustomModel, 'get_leaf_indices', XGBRegressor.apply)

In [20]:
# Verify that the model wrapper indeed has the desired attribute
CustomModel.get_leaf_indices

<function xgboost.sklearn.XGBModel.apply(self, X: Any, ntree_limit: int = 0, iteration_range: Optional[Tuple[int, int]] = None) -> numpy.ndarray>

In [21]:
model = CustomModel()
model.fit(
    X_train,
    y_train,
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [22]:
leafsim_instance = LeafSim(model)

In [23]:
top_n_ids, top_n_similarities = leafsim_instance.generate_explanations(
    X_train[feature_cols], X_test[feature_cols]
)

...

# Further resources

For a more comprehensive usage example, please refer to this blog post [https://datascience.ch/leafsim/](https://datascience.ch/leafsim/) and the corresponding [notebook](https://renkulab.io/projects/lucas.chizzali/leafsim/files/blob/notebooks/LeafSim.ipynb).